In [2]:
import torch

In [15]:
causal_input_ids = torch.load('causal_input_ids')
causal_prefill_res = torch.load('causal_prefill_res')
causal_logits = causal_prefill_res[0, -1, :]
print(causal_input_ids)
print(causal_input_ids.dtype)
print(causal_prefill_res.shape)
print(causal_prefill_res.dtype)
print(causal_logits.shape)

tensor([[151331, 151333,   3838,    374,   5538,   6832,     30]],
       device='cuda:0')
torch.int64
torch.Size([1, 7, 151552])
torch.float16
torch.Size([151552])


In [3]:
flash_input_ids = torch.load('flash_input_ids')
flash_prefill_res = torch.load('flash_prefill_out')
flash_logits = flash_prefill_res[-1]
print(flash_input_ids)
print(flash_prefill_res.shape)
print(flash_prefill_res.dtype)

tensor([    1,  1724,   338,  6483,  6509, 29973], device='cuda:0')
torch.Size([6, 32000])
torch.float16


In [30]:
flashinfer_input_ids = torch.load('flashinfer_input_ids')
flashinfer_prefill_out = torch.load('flashinfer_prefill_out')
flashinfer_logits = flashinfer_prefill_out[-1]
print(flashinfer_input_ids)
print(flashinfer_prefill_out.shape)
print(flashinfer_prefill_out.dtype)

tensor([151331, 151333,   3838,    374,   5538,   6832,     30],
       device='cuda:0')
torch.Size([7, 151552])
torch.float16


In [6]:
def assert_close(a, b, rtol, atol):
    # rtol, atol = {
    #     torch.float16: (1e-2, 5e-2),
    #     torch.bfloat16: (8e-3, 8e-3),
    # }[a.dtype]
    torch.testing.assert_close(a, b, rtol=rtol, atol=atol)

In [31]:
assert_close(causal_logits, flashinfer_logits, 1e-1, 3e-2)

AssertionError: Tensor-likes are not close!

Mismatched elements: 138115 / 151552 (91.1%)
Greatest absolute difference: 15.4140625 at index (198,) (up to 0.03 allowed)
Greatest relative difference: inf at index (84556,) (up to 0.1 allowed)

In [20]:
import torch.nn.functional as F

def calc_divergence(logits1, logits2):
    sm1 = F.softmax(logits1, dim=1)
    sm2 = F.softmax(logits2, dim=1)
    kl_divergence_elementwise = F.kl_div(F.log_softmax(sm1, dim=1), sm2, reduction='none')
    return torch.sum(kl_divergence_elementwise, dim=1)/sm1.size(1)


def calc_divergence_single(sm1, sm2):
    return F.kl_div(F.log_softmax(sm1), sm2, reduction='batchmean')

def calc_divergence_single_log(logits1, logits2):
    sm1 = F.softmax(logits1)
    sm2 = F.softmax(logits2)
    return F.kl_div(F.log_softmax(sm1), sm2, reduction='batchmean')

# flash_prefill_sm = F.softmax(flash_prefill_res, dim=1)
# flashinfer_prefill_sm = F.softmax(flashinfer_prefill_out, dim=1)
# kl_divergence_elementwise = F.kl_div(F.log_softmax(flash_prefill_sm, dim=1), flashinfer_prefill_sm, reduction='none')
# divergence = torch.sum(kl_divergence_elementwise, dim=1)/flash_prefill_sm.size(1)
# print(divergence)


In [33]:
sm1 = F.softmax(causal_logits)
sm2 = F.softmax(flashinfer_logits)

/tmp/ipykernel_28828/3079652309.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sm1 = F.softmax(causal_logits)
/tmp/ipykernel_28828/3079652309.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sm2 = F.softmax(flashinfer_logits)


In [34]:
assert_close(sm1, sm2, 1e-2, 3e-3)

AssertionError: Tensor-likes are not close!

Mismatched elements: 30 / 151552 (0.0%)
Greatest absolute difference: 0.51220703125 at index (198,) (up to 0.003 allowed)
Greatest relative difference: 10232.0 at index (198,) (up to 0.01 allowed)

In [35]:
sm1[[198, 8303]], sm2[[198, 8303]]

(tensor([0.5122, 0.0469], device='cuda:0', dtype=torch.float16),
 tensor([5.0068e-05, 1.8001e-05], device='cuda:0', dtype=torch.float16))

In [36]:
sm1.max(), sm2.max()

(tensor(0.5122, device='cuda:0', dtype=torch.float16),
 tensor(0.2363, device='cuda:0', dtype=torch.float16))

In [37]:
calc_divergence_single_log(causal_logits, flashinfer_logits)

/tmp/ipykernel_28828/3746471026.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sm1 = F.softmax(logits1)
/tmp/ipykernel_28828/3746471026.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sm2 = F.softmax(logits2)
/tmp/ipykernel_28828/3746471026.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.kl_div(F.log_softmax(sm1), sm2, reduction='batchmean')


tensor(4.3690e-05, device='cuda:0', dtype=torch.float16)

In [91]:
ts1 = torch.Tensor([0.1, 0.85, 0.05])
ts2 = torch.Tensor([0.8, 0.15, 0.05])
calc_divergence_single(ts1, ts2)

/tmp/ipykernel_15778/2316408785.py:11: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.kl_div(F.log_softmax(sm1), sm2, reduction='batchmean')


tensor(0.2268)

In [79]:
torch.Tensor([0.1, 0.8, 0.1]).repeat(2, 1)

tensor([[0.1000, 0.8000, 0.1000],
        [0.1000, 0.8000, 0.1000]])